<a href="https://colab.research.google.com/github/Sivashankar7777/Major_Project_2025/blob/main/Table_using_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python numpy pandas pytesseract ultralyticsplus
!apt install -y tesseract-ocr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.1/699.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,682 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
import os


In [ ]:
import numpy as np
import pandas as pd
import pytesseract
from pytesseract import Output
from ultralyticsplus import YOLO
from PIL import Image  # pillow
import cv2
import subprocess
import csv
import os


# load The image
image = cv2.imread('./test3.jpg')
img = Image.fromarray(image)

# load model
model = YOLO('keremberke/yolov8m-table-extraction')

# set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # maximum number of detections per image

# perform inference
results = model.predict(img)

tables_vesul = []
tables = []

for i in range(len(results[0].boxes.data.numpy())):
    x1, y1, x2, y2, _, _ = tuple(int(item) for item in results[0].boxes.data.numpy()[i])  # (96, 586, 947, 1286)
    x2 += 10
    y2 += 10
    x1 -= 10
    y1 -= 10

    # cropping
    cropped_image = image[y1:y2, x1:x2]
    print(cropped_image.shape)
    tables.append(cropped_image)
    image_tab = Image.fromarray(cropped_image)
    tables_vesul.append(image_tab)

# Step 2 : Extracting data from the table


def add_10_percent_padding(img):
    image_height = img.shape[0]
    padding = int(image_height * 0.1)
    padded_img = cv2.copyMakeBorder(img, padding, padding, padding, padding, cv2.BORDER_CONSTANT,
                                     value=[255, 255, 255])
    return padded_img


def img_Preprocessing(img):
    # Grey-scaling
    grayscale_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # converting it to binary image by Thresholding
    thresholded_image = cv2.threshold(grayscale_image, 127, 255, cv2.THRESH_BINARY)[1]
    # Inverting
    inverted_image = cv2.bitwise_not(thresholded_image)
    return inverted_image


def img_Preprocessing_ocr(img):
    # Thresholding
    thresholded_image = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
    # Inverting
    # inverted_image = cv2.bitwise_not(thresholded_image)
    return thresholded_image


# Combining Vertical And Horizontal Lines
def combine_eroded_images(img):
    # Eroding Vertical Lines
    hor = np.array([[1, 1, 1, 1, 1, 1]])
    ver_erode_img = cv2.erode(img, hor, iterations=5)
    ver_dilate_img = cv2.dilate(ver_erode_img, hor, iterations=10)
    # Eroding Horizontal Lines
    ver = np.array([[1], [1], [1], [1], [1], [1], [1]])
    hor_erode_img = cv2.erode(img, ver, iterations=5)
    hor_dilate_img = cv2.dilate(hor_erode_img, ver, iterations=10)
    # Combining
    combined_image = cv2.add(ver_dilate_img, hor_dilate_img)
    # dilate combined_image to make lines thicker
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    combined_image_dilated = cv2.dilate(combined_image, kernel, iterations=5)
    return combined_image_dilated


# Removing The Lines
def subtract_combined_and_dilated_image_from_original_image(processed_img, combined_image_dilated):
    img_without_lines = cv2.subtract(processed_img, combined_image_dilated)
    # remove noise with erode and dilate
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    image_without_lines_noise_removed = cv2.erode(img_without_lines, kernel)
    image_without_lines_noise_removed = cv2.dilate(image_without_lines_noise_removed, kernel)
    return image_without_lines_noise_removed


# Finding The Blobs and Extracting The Text From Them

# Use Dilation To Convert The Words Into Blobs
def dilate_image(thresholded_image):
    kernel_to_remove_gaps_between_words = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                                                      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
    dilated_image = cv2.dilate(thresholded_image, kernel_to_remove_gaps_between_words, iterations=5)
    simple_kernel = np.ones((5, 5), np.uint8)
    dilated_image = cv2.dilate(dilated_image, simple_kernel, iterations=2)
    return dilated_image


# Find The Contours Of The Blobs
def find_contours(dilated_image, original_image):
    result = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = result[0]
    # The code below is for visualization purposes only.
    image_with_contours_drawn = original_image.copy()
    cv2.drawContours(image_with_contours_drawn, contours, -1, (0, 255, 0), 3)
    return contours, image_with_contours_drawn


# Convert The Blobs Into Bounding Boxes
def convert_contours_to_bounding_boxes(contours, original_image):
    bounding_boxes = []
    image_bounding_boxes = original_image.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, w, h))
        # The code below is for visualization purposes only.
        image_bounding_boxes = cv2.rectangle(image_bounding_boxes, (x, y), (x + w, y + h), (0, 255, 0), 5)
    return bounding_boxes, image_bounding_boxes


# Sorting The Bounding Boxes By X And Y Coordinates To Make Rows And Columns
def get_mean_height_of_bounding_boxes(bounding_boxes):
    heights = []
    for bounding_box in bounding_boxes:
        x, y, w, h = bounding_box
        heights.append(h)
    return np.mean(heights)


def sort_and_club_all_bounding_boxes_by_similar_y_coordinates_into_rows(bounding_boxes, mean_height):
    # sort_bounding_boxes_by_y_coordinate
    bounding_boxes = sorted(bounding_boxes, key=lambda x: x[1])
    # club_all_bounding_boxes_by_similar_y_coordinates_into_rows
    rows = []
    half_of_mean_height = mean_height / 2
    current_row = [bounding_boxes[0]]
    for bounding_box in bounding_boxes[1:]:
        current_bounding_box_y = bounding_box[1]
        previous_bounding_box_y = current_row[-1][1]
        distance_between_bounding_boxes = abs(current_bounding_box_y - previous_bounding_box_y)
        if distance_between_bounding_boxes <= half_of_mean_height:
            current_row.append(bounding_box)
        else:
            rows.append(current_row)
            current_row = [bounding_box]
    rows.append(current_row)

    # sort_all_rows_by_x_coordinate
    for row in rows:
        row.sort(key=lambda x: x[0])
    # Sort rows by the Y coordinate of the first box in each row
    # rows.sort(key=lambda row: row[0][1])
    return rows


# Extracting The Text From The Bounding Boxes Using OCR
def crop_each_bounding_box_and_ocr(rows, original_image):
    table = []
    current_row = []
    image_number = 0
    first_row = True  # Flag to indicate the first row

    for row in rows:
        if first_row:
            # Add an empty string at the beginning of the first row
            current_row.append(' ')  # Space character inside quotes
            first_row = False  # Reset the flag

        for bounding_box in row:
            x, y, w, h = bounding_box
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
            image_without_lines_noise_removed = cv2.erode(add_10_percent_padding(original_image[y:y + h, x:x + w]),
                                                           kernel)
            cropped_image = cv2.dilate(image_without_lines_noise_removed, kernel)
            image_slice_path = "img_" + str(image_number) + ".jpg"
            cv2.imwrite(image_slice_path, cropped_image)
            results_from_ocr = tesseract_results(image_slice_path)
            if not results_from_ocr:
                current_row.append('""')  # Add empty quotes if OCR result is empty
            else:
                current_row.append(results_from_ocr)
            image_number += 1
        table.append(current_row)
        current_row = []
    return table



def tesseract_results(image_path):
    # Construct the Tesseract OCR command
    tesseract_command = (
        'tesseract ' + image_path + ' - -l eng --oem 3 --psm 6 --dpi 300 '
    )

    # Run the Tesseract command using subprocess
    output = subprocess.getoutput(tesseract_command)

    # Strip any leading or trailing whitespace from the output
    output = output.strip('|')
    output = output.replace('\n', ' ')
    output = output.strip()
    return output


def generate_csv_file(input_table, i):
    output_dir = "./outputs"
    os.makedirs(output_dir, exist_ok=True)

    with open(f"{output_dir}/Extracted_table_{i}.csv", 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for row in input_table:
            csv_writer.writerow(row)

    print(f"Combined Table {i}:")
    for row in input_table:
        print(row)


for i, table in enumerate(tables):
    image_with_padding = add_10_percent_padding(table)
    processed_img = img_Preprocessing(image_with_padding)
    combined_img = combine_eroded_images(processed_img)
    img_without_lines = subtract_combined_and_dilated_image_from_original_image(processed_img, combined_img)
    # Finding the cells & extracting the text using OCR
    dilated_img = dilate_image(img_without_lines)
    contours, image_with_contours_drawn = find_contours(dilated_img, image_with_padding)
    bounding_boxes, img_bouding_boxes = convert_contours_to_bounding_boxes(contours, image_with_padding)
    mean_height = get_mean_height_of_bounding_boxes(bounding_boxes)
    sorted_rows = sort_and_club_all_bounding_boxes_by_similar_y_coordinates_into_rows(bounding_boxes, mean_height)
    table = crop_each_bounding_box_and_ocr(sorted_rows, image_with_padding)
    generate_csv_file(table, i)

100%|██████████| 6.23M/6.23M [00:00<00:00, 74.2MB/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/nn/tasks.py:634: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  retur

config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/52.0M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/ultralytics/nn/tasks.py:634: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location="cpu"), file  # load



0: 640x448 1 bordered, 1940.4ms
Speed: 23.8ms preprocess, 1940.4ms inference, 36.6ms postprocess per image at shape (1, 3, 640, 448)
(641, 1063, 3)
Combined Table 0:
[' ', 'New version', 'Past Versions']
['Restore fonts', 'Yes', 'No']
['Restore font size', 'Yes', 'No']
['Restore stamps', 'Yes', 'No']
['Restore tables', 'Yes', 'No']
['Restore pictures', 'Yes', 'No']
['Preview the results', 'Yes', 'No']
['Edit freely', 'Yes', 'No']
